## _FC_ AutoEncoder with Sedov Data

In [2]:
%matplotlib inline

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
# f = open('sedov-pres.dat','rb')
root = '/Users/jinzhenwang/PhD/Research_HPC/AE_compression/'

f = open(os.path.join(root,'data/GROMACS_data/data_100step_1000seg/md_0_seg.txt'),'r')
input_data = f.readlines()
# data_node = tf.placeholder(tf.string)
# record_bytes = tf.decode_raw(data_node, tf.float64)
# sess = tf.Session()
# data_num = sess.run(record_bytes, {data_node: input_data})
for i in range(1,100):
    f = open(os.path.join(root,'data/GROMACS_data/data_100step_1000seg/md_'+str(i)+'_seg.txt'),'r')
    input_data = np.concatenate([input_data,f.readlines()],0)

# print(np.shape(input_data))
# print(input_data[1000])
input_data = [float(line.split(' ')[1]) for line in input_data]
# print(np.shape(input_data))

In [4]:
# print(np.shape(input_data))
# print(input_data)
input_data = np.reshape(input_data,[10000,1000])
print(np.shape(input_data))

(10000, 1000)


In [5]:
# img = mnist.train.images[2]
# plt.imshow(img.reshape((28, 28)), cmap='Greys_r')
# input_data = np.reshape(input_data,[10000,1000,1])

In [6]:
# Size of the encoding layer (the hidden layer)
# encoding_dim = 32 # feel free to change this value

# Input and target placeholders
inputs_ = tf.placeholder(tf.float32, (None,1000))
targets_ = tf.placeholder(tf.float32, (None,1000))

# Output of hidden layer, single fully connected layer here with ReLU activation
encoded1 = tf.layers.dense(inputs_, 512, activation = tf.nn.relu)
encoded2 = tf.layers.dense(encoded1, 256, activation = tf.nn.relu)
encoded3 = tf.layers.dense(encoded2, 128, activation = tf.nn.relu)
encoded4 = tf.layers.dense(encoded3, 64, activation = tf.nn.relu)
encoded5 = tf.layers.dense(encoded4, 32, activation = tf.nn.relu)

encoded6 = tf.layers.dense(encoded5, 64, activation = tf.nn.relu)
encoded7 = tf.layers.dense(encoded6, 128, activation = tf.nn.relu)
encoded8 = tf.layers.dense(encoded7, 256, activation = tf.nn.relu)
encoded9 = tf.layers.dense(encoded8, 512, activation = tf.nn.relu)
logits = tf.layers.dense(encoded9, 1000, activation=None)
# Sigmoid output from logits
# decoded = tf.nn.sigmoid(logits)
encoded = encoded5
decoded = logits
# Sigmoid cross-entropy loss
# loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=targets_, logits=logits)
loss = tf.pow((targets_-decoded),2)
# Mean of the loss
cost = tf.reduce_mean(loss)
opt = tf.train.AdamOptimizer(0.0001).minimize(cost)

In [7]:
print(inputs_)

Tensor("Placeholder:0", shape=(?, 1000), dtype=float32)


## Training

In [8]:
# Create the session
sess = tf.Session()

In [9]:
epochs = 10000
# batch_size = 200
sess.run(tf.global_variables_initializer())


In [ ]:
for e in range(epochs):
    feed = {inputs_: input_data, targets_: input_data}
#     for ii in range(mnist.train.num_examples//batch_size):
#         batch = mnist.train.next_batch(batch_size)
#         feed = {inputs_: batch[0], targets_: batch[0]}
    batch_cost, _ = sess.run([cost, opt], feed_dict=feed)
    if(e % 100 == 0):
        print("Epoch: {}/{}...".format(e+1, epochs),
              "Training loss: {:.4f}".format(batch_cost))

Epoch: 1/10000... Training loss: 13.0545
Epoch: 101/10000... Training loss: 0.0030
Epoch: 201/10000... Training loss: 0.0010
Epoch: 301/10000... Training loss: 0.0010


## Checking out the results

Below I've plotted some of the test images along with their reconstructions. For the most part these look pretty good except for some blurriness in some parts.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(20,4))
in_imgs = input_data[:10]
# print(np.shape(in_imgs))
reconstructed, compressed = sess.run([decoded, encoded], feed_dict={inputs_: in_imgs})

for images, row in zip([in_imgs, reconstructed], axes):
    for img, ax in zip(images, row):
        ax.imshow(img.reshape((100, 10)), cmap='Greys_r')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

fig.tight_layout(pad=0.1)

In [ ]:
reconstructed

In [ ]:
in_imgs